In [1]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import joblib
import threading

In [2]:
# Load dataset
file_path = 'churn.csv'  # Adjust this path if necessary
df = pd.read_csv(file_path)
print("Dataset loaded successfully.")
print(f"Dataset shape: {df.shape}")

Dataset loaded successfully.
Dataset shape: (1000, 12)


In [4]:

# Ensure the target column is correctly identified
target_column = 'Exited'  # Replace this with your target column name if it's different
if target_column not in df.columns:
    raise ValueError(f"Target column '{target_column}' not found in dataset.")


In [5]:
# Preprocessing
df.fillna(df.select_dtypes(include='number').mean(), inplace=True)  # Fill missing values for numeric columns
df = pd.get_dummies(df, drop_first=True)  # One-hot encode categorical variables


In [6]:
# Split features and target
X = df.drop(columns=[target_column])
y = df[target_column]

In [7]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:

# Train Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [9]:
# Evaluate initial model
y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)[:, 1]
metrics = {
    "Accuracy": accuracy_score(y_test, y_pred),
    "Precision": precision_score(y_test, y_pred),
    "Recall": recall_score(y_test, y_pred),
    "F1 Score": f1_score(y_test, y_pred),
    "ROC AUC Score": roc_auc_score(y_test, y_proba)
}

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
# Grid Search for Hyperparameter Tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [10, 20, None],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200]},
             scoring='accuracy')

In [13]:

# Best model
best_rf_model = grid_search.best_estimator_

In [14]:
# Evaluate the best model
y_pred_best = best_rf_model.predict(X_test)
y_proba_best = best_rf_model.predict_proba(X_test)[:, 1]
metrics_best = {
    "Accuracy": accuracy_score(y_test, y_pred_best),
    "Precision": precision_score(y_test, y_pred_best),
    "Recall": recall_score(y_test, y_pred_best),
    "F1 Score": f1_score(y_test, y_pred_best),
    "ROC AUC Score": roc_auc_score(y_test, y_proba_best)
}

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
# Save the best model
joblib.dump(best_rf_model, 'best_rf_churn_model.pkl')


['best_rf_churn_model.pkl']

In [16]:

# Feature Importance
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

In [19]:
# Tkinter GUI
def show_results():
    root = tk.Tk()
    root.title("Churn Prediction Results")

    # Display evaluation metrics
    metrics_text = "\n".join([f"{metric}: {score:.4f}" for metric, score in metrics_best.items()])
    metrics_label = tk.Label(root, text=f"--- Tuned Model Metrics ---\n{metrics_text}", font=("Arial", 12))
    metrics_label.pack(pady=10)

    # Display top features
    feature_text = "\n".join([f"{i+1}. {feature}: {importance:.4f}" 
                              for i, (feature, importance) in enumerate(feature_importances.head(5).values)])
    features_label = tk.Label(root, text=f"--- Top Features Driving Churn ---\n{feature_text}", font=(" Arial", 12))
    features_label.pack(pady=10)

    # Button for suggested actions
    actions_text = "\n".join([f"- Focus on '{feature}'." for feature in feature_importances.head(5)['Feature']])
    actions_button = tk.Button(root, text="Show Suggested Actions", command=lambda: show_message(actions_text))
    actions_button.pack(pady=10)

    root.mainloop()

def show_message(actions_text):
    messagebox.showinfo("Suggested Actions", actions_text)

# Function to run the GUI in a separate thread
def run_gui():
    show_results()

# Start the GUI in a new thread
threading.Thread(target=run_gui).start()